In [7]:
import json
import numpy as np
import wandb
import re

wandb.init(project='EunbiNet_prototype')
config = wandb.config

wandb: wandb version 0.11.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [3]:
from util import *

In [5]:
json

<module 'json' from 'C:\\Users\\hoang\\Anaconda3\\envs\\Torch\\lib\\json\\__init__.py'>

In [ ]:
#Loading the data
W_norm,vocab,ivocab = load_glove()
    
config.batch_size = 32
batch_size = config.batch_size

train_data = load_training_batch()

pad_tensor = np.zeros(100)
unk_tensor = np.ones(100)

train_embedding = get_embedding(training_data)

#unit test 
debug_set = train_embedding
print(debug_set.shape)

In [3]:


#Define the architecture
import torch
import torch.nn as nn

class EunbiNet(nn.Module):
    def __init__(self,context_length):
        super().__init__()
        self.n = context_length*2
        self.conv1d1 = nn.Conv1d(self.n,16,1,stride=1)
        self.conv1d2 = nn.Conv1d(16,8,1,stride=1)
        self.conv1d3 = nn.Conv1d(8,4,1,stride=1)
        self.conv1d4 = nn.Conv1d(4,1,1,stride=1)
        self.tanh = nn.Tanh()
        
    def forward(self,text):
        s = self.conv1d1(text)
        s = self.tanh(s)
        s = self.conv1d2(s)
        s = self.tanh(s)
        s = self.conv1d3(s)
        s = self.tanh(s)
        s = self.conv1d4(s)
        s = self.tanh(s)
        return s



In [34]:
import torch.optim as optim


config.lr = 0.0005
config.momentum = 0.005
optimizer = optim.SGD(model.parameters(),lr=config.lr,momentum=config.momentum,weight_decay=0.01)
criterion = nn.L1Loss()

#scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, debug_set.shape[0], eta_min=config.lr)
#learning rate adjustment -- try 0.001

In [35]:
target_label = W_norm[vocab['cat']]

In [36]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    for batch in iterator:
        optimizer.zero_grad()
        batch_size = batch.shape[0]
        #print(batch)
        predictions = model(torch.Tensor(batch)).squeeze(1)
        labels = torch.Tensor(np.array([target_label for _ in range(batch_size)]))
        y = torch.Tensor(np.ones((1,32)))
        #print(predictions.shape)
        #print(labels.shape)
        loss = criterion(predictions,labels)
        ##print(loss)
        
        loss.backward()
        
        optimizer.step()
        epoch_loss += loss.item()
        
    return epoch_loss 

In [37]:
config.epochs = 50

best_valid_loss = float('inf')



for epoch in range(config.epochs):   
    train_loss = train(model,iter(debug_set), optimizer, criterion)

    #epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    #if valid_loss < best_valid_loss:
     #   best_valid_loss = valid_loss
      #  torch.save(model.state_dict(), 'tut1-model.pt')
    
    #print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    wandb.log({"loss":train_loss})
    print(f'\tTrain Loss: {train_loss:.3f}')
    
    
    #we can cosine sim

	Train Loss: 96.586
	Train Loss: 85.739
	Train Loss: 75.654
	Train Loss: 66.382
	Train Loss: 58.122
	Train Loss: 50.903
	Train Loss: 44.563
	Train Loss: 39.042
	Train Loss: 34.213
	Train Loss: 29.912
	Train Loss: 26.117
	Train Loss: 22.873
	Train Loss: 20.196
	Train Loss: 18.000
	Train Loss: 16.197
	Train Loss: 14.724
	Train Loss: 13.512
	Train Loss: 12.504
	Train Loss: 11.656
	Train Loss: 10.930
	Train Loss: 10.295
	Train Loss: 9.731
	Train Loss: 9.226
	Train Loss: 8.774
	Train Loss: 8.369
	Train Loss: 8.007
	Train Loss: 7.682
	Train Loss: 7.389
	Train Loss: 7.121
	Train Loss: 6.874
	Train Loss: 6.647
	Train Loss: 6.435
	Train Loss: 6.237
	Train Loss: 6.052
	Train Loss: 5.879
	Train Loss: 5.716
	Train Loss: 5.562
	Train Loss: 5.417
	Train Loss: 5.278
	Train Loss: 5.146
	Train Loss: 5.020
	Train Loss: 4.898
	Train Loss: 4.782
	Train Loss: 4.670
	Train Loss: 4.562
	Train Loss: 4.458
	Train Loss: 4.357
	Train Loss: 4.260
	Train Loss: 4.165
	Train Loss: 4.074


In [38]:
#sample_test = 'tuna salmon mouse catch jump orange cheshire fur meow paw toebean shorthair food eat play british fish eat <pad> <pad>'
# try to sum across the whole set


num_batch = debug_set.shape[0]
sample_tensor = torch.Tensor([[get_glove_vec(word) for word in s.split(' ')]])
sample_output = model(sample_tensor)

output = sample_output.squeeze(1)
vec_output = output.detach().numpy()
vec_output.shape

def __distance(W, vocab, ivocab, vec_output):


    dist = np.dot(W, vec_output.T).squeeze(1)
    print(dist.shape)
    a = np.argsort(-dist)[:1000]

    print("\n                               Word       Cosine distance\n")
    print("---------------------------------------------------------\n")
    for i,x in enumerate(a):
        print("%d%35s\t\t%f" % (i,ivocab[str(x)], dist[x]))


__distance(W_norm,vocab,ivocab,vec_output)

def cosim(v1,v2):
    return np.dot(v1,v2)

cosim(vec_output,target_label)

(400000,)

                               Word       Cosine distance

---------------------------------------------------------

0                                cat		1.019540
1                                dog		0.894930
2                             rabbit		0.757930
3                               cats		0.749405
4                             monkey		0.740772
5                                pet		0.737548
6                               dogs		0.729291
7                              mouse		0.703127
8                              puppy		0.691955
9                                rat		0.679731
10                             spider		0.660956
11                           elephant		0.653884
12                               bird		0.642017
13                                pig		0.640112
14                              snake		0.638292
15                               baby		0.637612
16                                boy		0.636480
17                             animal		0.634454
18               

array([1.01953965])